In [1]:
import requests
import lxml.html as lh
import pandas as pd
import re

In [2]:
#This code is inspired in https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059
def parse_url(url, date):
    #Create a handle, page, to handle the contents of the website
    page = requests.get(url)

    #Store the contents of the website under doc
    doc = lh.fromstring(page.content)

    #Parse data that are stored between <tr>..</tr> of HTML
    #tr_elements = doc.xpath('//tr')
    tr_elements = doc.xpath('/html/body/div[1]/div[4]/div[1]/div/div[3]/table/tbody/tr')
        
    
    #Create empty list
    col=[]
    i=0#For each row, store each first element (header) and an empty list
    for t in tr_elements[0]:
        i+=1
        name=t.text_content().strip()
        #print ("column:", i, name)
        col.append((name,[]))
        
    #Append the date column   
    col.append(("Fecha", []))
        
    #Since out first row is the header, data is stored on the second row onwards
    for j in range(1,len(tr_elements)):
        #print ("row:", j)
        
        #T is our j'th row
        T=tr_elements[j]
        #print ("row:", T)

        #If row is not of size 10, the //tr data is not from our table 
        if len(T)!=2:
            break

        #i is the index of our column
        i=0

        #Iterate through each element of the row
        for t in T.iterchildren():
            #print("i:", i, "j:", j, "cell:", t)
            data=t.text_content().strip() 
            #print("i:", i, "j:", j, "data:", data)
            #Check if row is empty
            if i>0:
            #Convert any numerical value to integers
                try:
                    data=int(data)                    
                except:
                    pass
            #Append the data to the empty list of the i'th column
            #print (data)
            col[i][1].append(data)
            #Increment i for the next column
            i+=1    
            
        #Append the date column   
        col[i][1].append(date)
            
    #print("col:", col)        
    
    Dict={title:column for (title,column) in col}    
    df=pd.DataFrame(Dict)
    #df.head()
    
    return df


In [21]:
#!pip install PyPDF2
def parse_pdf_url(url, date):
    import PyPDF2

    response = requests.get(url)
    my_raw_data = response.content

    with open("my_pdf.pdf", 'wb') as my_data:
        my_data.write(my_raw_data)

    page_content = ""
        
    read_pdf = PyPDF2.PdfFileReader("my_pdf.pdf")
    number_of_pages = read_pdf.getNumPages()
    #page = read_pdf.getPage(0)
    #page_content = page.extractText()
    for page_number in range(number_of_pages):   # use xrange in Py2
            page = read_pdf.getPage(page_number)
            page_content = page_content + page.extractText()
            
    page_content = clean_pdf_content(page_content)        
            
    page_content_split = page_content.split('\n \n')
    type(page_content_split)
        
    page_content_split_filtered = list(filter(lambda x: "nosticados" not in x and "notificados" not in x and "Zona Básica" not in x and x != "Casos" and x != " " and x != "" and x != " \n" and not "/2020" in x, page_content_split))
    print(page_content_split_filtered)

    elements = int((len(page_content_split_filtered))/2)
    #print(elements)
    type(elements)

    import  numpy as np
    reshaped_array = np.reshape(np.array(page_content_split_filtered), (elements, 2))

    a_dataframe = pd.DataFrame(reshaped_array, columns=["Zona Básica", "Casos"])
    a_dataframe = a_dataframe.astype({'Casos': 'int32'})

    a_dataframe = a_dataframe.assign(Fecha=[date]*elements)
    
    return a_dataframe

def clean_pdf_content(page_content):    
        
    page_content = page_content.replace("1\n \n \nCasos\n \nnotificados", "Casos notificados") 
            
    page_content = page_content.replace("TORRERO\n \nLA PAZ", "TORRERO LA PAZ")
    page_content = page_content.replace("AVENIDA \nCATALUÑA", "AVENIDA CATALUÑA")
    page_content = page_content.replace("BIESCAS\n-\nVALLE DE TENA", "BIESCAS-VALLE DE TENA ")
    page_content = page_content.replace("SANTA \nEULALIA DEL CAMPO", "SANTA EULALIA DEL CAMPO")
    page_content = page_content.replace("VALDESPARTERA\n-\nMONTECANAL", "VALDESPARTERA-MONTECANAL")
    page_content = page_content.replace("SAGASTA\n-\nRUISEÑORES", "SAGASTA-RUISEÑORES")
    page_content = page_content.replace("HUESCA CAPITAL -1 \n(PERPETUO SOCORRO", "HUESCA CAPITAL Nº 1 (PERPETUO SOCORRO)")
    page_content = page_content.replace("HUESCA CAPITAL Nº 1 \n(PERPETUO SOCORRO", "HUESCA CAPITAL Nº 1 (PERPETUO SOCORRO)")
    page_content = page_content.replace("HUESCA CAPITAL Nº 2 (SANTO \nGRIAL)", "HUESCA CAPITAL Nº 2 (SANTO GRIAL)")
    page_content = page_content.replace("MADRE \nVEDRUNA\n-\nMIRAFLORES", "MADRE VEDRUNA-MIRAFLORES")
    page_content = page_content.replace("MADRE VEDRUNA\n-\nMIRAFLORES", "MADRE VEDRUNA-MIRAFLORES") 
    page_content = page_content.replace("FERNANDO EL\n \nCATOLICO", "FERNANDO EL CATOLICO")    
    page_content = page_content.replace("SAN JOSE\n \nNORTE", "SAN JOSE NORTE")
    
    page_content = page_content.replace("\n-\n \n", "-")
    page_content = page_content.replace("Nº\n \n", "-")
        
    page_content = re.sub(r'(\d+)\n \n\d+', r'\1', page_content)
    
    print(page_content)
        
    return page_content

In [15]:
#Scrap the HTML as data frames
frames = [parse_url("http://www.aragonhoy.net/index.php/mod.noticias/mem.detalle/relmenu.9/id.263522", "2020-07-21"), 
          parse_url("http://www.aragonhoy.net/index.php/mod.noticias/mem.detalle/relmenu.9/id.263587", "2020-07-22"),
          parse_pdf_url("http://www.aragonhoy.net/index.php/mod.documentos/mem.descargar/fichero.documentos_zonas_de_salud_240720_b207d97a%232E%23pdf", "2020-07-23"),
          parse_pdf_url("http://www.aragonhoy.net/index.php/mod.documentos/mem.descargar/fichero.documentos_Casos_diagnosticados_el_24_21d693c5%232E%23pdf", "2020-07-24"),
          parse_pdf_url("http://www.aragonhoy.net/index.php/mod.documentos/mem.descargar/fichero.documentos_Casos_diagnosticados_a_25_f6c24e2b%232E%23pdf", "2020-07-25"),              
          parse_pdf_url("http://www.aragonhoy.net/index.php/mod.documentos/mem.descargar/fichero.documentos_Casos_notificados_el_27_f174cec5%232E%23pdf", "2020-07-26"),
          parse_pdf_url("http://www.aragonhoy.net/index.php/mod.documentos/mem.descargar/fichero.documentos_Casos_notificados_el_28_502913f8%232E%23pdf", "2020-07-27"),
          parse_pdf_url("http://www.aragonhoy.net/index.php/mod.documentos/mem.descargar/fichero.documentos_Casos_notificados_el_29_8534c52d%232E%23pdf", "2020-07-28"),
          parse_pdf_url("http://www.aragonhoy.net/index.php/mod.documentos/mem.descargar/fichero.documentos_Casos_notificados_a_30_11f0e64d%232E%23pdf", "2020-07-29")]

#Concatenate the data frames
result = pd.concat(frames)

result.head()

#Pivot the table
table = pd.pivot_table(result, values='Casos', index=['Zona Básica'],
                    columns=['Fecha'], fill_value=0)

#print(table)
    

Zona Básica
 
Casos
 
DELICIAS NORTE
 
15
 
HERNAN CORTES
 
15
 
AVENIDA CATALUÑA
 
11
 
FERNANDO EL CATOLICO
 
11
 
SAGASTA-RUISEÑORES
 
9
 
SAN JOSE SUR
 
9
 
ALCAÑIZ
 
8
 
CASPE
 
8
 
DELICIAS SUR
 
8
 
FRAGA
 
8
 
CASETAS
 
7
 
MONZON URBANA
 
7
 
SAN PABLO
 
7
 
TERUEL CENTRO
 
7
 
ZARAGOZA
 
6
 
ACTUR OESTE
 
6
 
INDEPENDENCIA
 
6
 
MARIA DE HUERVA
 
6
 
TORRE RAMONA
 
6
 
UNIVERSITAS
 
6
 
ALBALATE DE CINCA
 
5
 
EJEA DE LOS CABALLEROS
 
5
 
HUESCA CAPITAL Nº 1 (PERPETUO SOCORRO))
 
5
 
LAS FUENTES NORTE
 
5
 
ROMAREDA -SEMINARIO
 
5
 
SAN JOSE CENTRO
 
5
 
TORRERO LA PAZ
 
5
 
ALIAGA
 
4
 
BINEFAR
 
4
 
CASABLANCA
 
4
 
MADRE VEDRUNA-MIRAFLORES
 
4
 
PARQUE GOYA
 
4
 
VENECIA
 
4
 
ACTUR NORTE
 
3
 
ACTUR SUR
 
3
 
ALAGON
 
3
 
ALMUDEVAR
 
3
 
HUESCA CAPITAL Nº 2 (SANTO GRIAL)
 
3
 
OLIVER
 
3
 
SANTA EULALIA DEL CAMPO
 
3
 
TAUSTE
 
3
 
TERUEL ENSANCHE
 
3
 
ZUERA
 
3
 
ARRABAL
 
2
 
BAGUENA
 
2
 
BOMBARDA
 
2
 
GRAÑEN
 
2
 
HIJAR
 
2
 
JACA
 
2
 
MEQUINENZA
 
2
 
MIRALBUENO
-

ValueError: cannot reshape array of size 141 into shape (70,2)

In [17]:
#Delete the TOTAL row, because we do not have total every day
table = table[table.index != "TOTAL"]
#Create the TOTAL row again
table = table.append(table.sum().rename('TOTAL'))

#Create the TOTAL column
total_column = table["2020-07-21"] + table["2020-07-22"] + table["2020-07-23"] + table["2020-07-24"] + table["2020-07-25"] + table["2020-07-26"] + table["2020-07-27"] + table["2020-07-28"]
table["TOTAL"] = total_column

#10 health districts with most cases
top_10 = table[table.index != "TOTAL"].sort_values(by=['TOTAL'], ascending=False).head(10)


NameError: name 'table' is not defined

In [16]:
#transpose the top 10
top_10_t = top_10.transpose()

NameError: name 'top_10' is not defined

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(rc={'figure.figsize':(11, 4)})

top_10_t[top_10_t.index != "TOTAL"].plot().legend(loc='center left',bbox_to_anchor=(1.0, 0.5))

In [20]:
parse_pdf_url("http://www.aragonhoy.net/index.php/mod.documentos/mem.descargar/fichero.documentos_Casos_notificados_a_30_11f0e64d%232E%23pdf", "2020-07-29")

Casos 
notificados 
a 30
/7/2020
 
 
 
Zona Básica
 
Casos
 
HUESCA RURAL
 
53
 
FRAGA
 
36
 
MEQUINENZA
 
32
 
AVENIDA CATALUÑA
 
15
 
PARQUE GOYA
 
14
 
DELICIAS NORTE
 
12
 
UNIVERSITAS
 
11
 
BOMBARDA
 
10
 
TERUEL CENTRO
 
10
 
VALDEFIERRO
 
9
 
TERUEL ENSANCHE
 
7
 
ACTUR OESTE
 
6
 
ALMOZARA
 
6
 
CASPE
 
6
 
DELICIAS SUR
 
6
 
MONZON URBANA
 
6
 
OLIVER
 
6
 
UTRILLAS
 
6
 
MIRALBUENO
-
GARRAPINILLOS
 
5
 
SANTA ISABEL
 
5
 
EJEA DE LOS CABALLEROS
 
4
 
UTEBO
 
4
 
VALDESPARTERA-MONTECANAL
 
4
 
CASETAS
 
3
 
SARRION
 
3
 
TAMARITE DE LITERA
 
3
 
TORRERO LA PAZ
 
3
 
VENECIA
 
3
 
ACTUR SUR
 
2
 
BARBASTRO
 
2
 
CASABLANCA
 
2
 
FERNANDO EL CATOLICO
 
2
 
GRAUS
 
2
 
HERNAN CORTES
 
2
 
INDEPENDENCIA
 
2
 
JACA
 
2
 
MARIA DE HUERVA
 
2
 
REBOLERIA
 
2
 
SAGASTA-RUISEÑORES
 
2
 
SAN PABLO
 
2
 
ALAGON
 
1
 
ALBALATE DE CINCA
 
1
 
ALFAJARIN
 
1
 
ANDORRA
 
1
 
ARRABAL
 
1
 
AYERBE
 
1
 
BAGUENA
 
1
 
BORJA
 
1
 
BROTO
 
1
 
BUJARALOZ
 
1
 
EPILA
 
1
 
GRAÑEN
 
1
 
HIJAR
 
1
 


ValueError: cannot reshape array of size 141 into shape (70,2)

In [ ]:
table_no_total = table[table.index != "TOTAL"].sort_values(by=['TOTAL'], ascending=False)
table_no_total_t =  table_no_total.transpose()

table_no_total_t[table_no_total_t.index != "TOTAL"].plot(stacked=True).legend(loc='center left',bbox_to_anchor=(1.0, 0.5))